In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# TODO: create seperate a file for preprocessed data??
df = pd.read_csv('../data/raw/average-monthly-surface-temperature.csv')
df.reset_index()
df = df.rename(columns={'Average surface temperature': 'Daily Average Temp', 'Average surface temperature.1': 'Monthly Average Temp', 'year' : 'Year'})

In [ ]:
df.sample(5)

In [ ]:
df.info()

In [ ]:
df.describe()

# Missing values

In [ ]:
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)

# there are no missing values

# Duplicates

In [ ]:
duplicate_rows = df[df.duplicated()]
print("Duplicate Rows:")
print(duplicate_rows)

# there are no duplicate rows

# Convert Data Types

In [ ]:
df["Day"] = pd.to_datetime(df["Day"], format='%Y-%m-%d')
df["Month"] = pd.to_datetime(df["Day"]).dt.month
df['Year'] = pd.to_datetime(df['Day']).dt.year

df.info()

In [ ]:
df.sample(10)

# Verifying Entity and Code columns

In [ ]:
for name, num_of_occurrences in df['Entity'].value_counts().items():
    # check if every country has the same number of entries
    if num_of_occurrences != 1020:
        print(name, "has", num_of_occurrences)

for name in df["Entity"].unique():
    # check if every country name (Entity column) matches the country code 
    correct_code = df.loc[df["Entity"] == name].iloc[0]["Code"]      
    incorrect_rows = df.loc[(df["Entity"] == name) & (df["Code"] != correct_code)]
    if incorrect_rows.shape[0] > 0:
          display(incorrect_rows)


# Data Visualization

In [ ]:
df

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)
plt.scatter(x='Month', y='Daily Average Temp', data=df)

In [ ]:
df.groupby(["Entity", "Year"], as_index=False)["Monthly Average Temp"].first()

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)
#colors = itertools.cycle(["r", "b", "g"])
plt.scatter(x='Year', y='Monthly Average Temp', data=df.groupby(["Entity", "Year"], as_index=False)["Monthly Average Temp"].first())


# Drop Unnecessary Columns

In [ ]:
# having both "Enity" and "Code" is redundant, so I'm dropping a column
df.drop("Code", axis=1, inplace=True, errors='ignore')
df

# Scale Data

# Encode Entity Column
Use One-Hot-Encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df['Entity'].unique()

In [ ]:
ohe = OneHotEncoder(sparse_output=False).set_output(transform='pandas')
ohe_transform = ohe.fit_transform(df[["Entity"]])
encoded_df = df.drop('Entity', axis=1) 
encoded_df = pd.concat([encoded_df, ohe_transform], axis=1)
encoded_df

# Export preprocessed dataset to new CSV file

In [ ]:
encoded_df.to_csv('../data/processed/preprocessed_data.csv', index=False)